In [1]:
%pip install -r requirements.txt

Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu117.html
Note: you may need to restart the kernel to use updated packages.


## Imports

In [4]:
import torch
import torch_sparse

In [5]:
!nvidia-smi

Thu Dec  7 16:39:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  On   | 00000000:04:00.0 Off |                    0 |
| N/A   47C    P0    56W / 300W |      0MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Parameters

In [17]:
num_neurons = 10000
synapses_per_neuron = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [24]:
# Create a dense vector
indices = []
values = []
for i in range(num_neurons):
    for j in range(synapses_per_neuron):
        indices.append(((torch.randint(0, num_neurons, (1,))), i))
        values.append((torch.randn(1)))
indices = torch.tensor(indices).t()
values = torch.tensor(values)
        
coo_matrix = torch.sparse_coo_tensor(indices, values, (num_neurons, num_neurons)).coalesce().to(device)
csr_matrix = coo_matrix.to_sparse_csr().to(device)
dense_matrix = coo_matrix.to_dense().to(device)
dense_vector = torch.randn(num_neurons).to(device)

In [32]:
%timeit torch.mv(coo_matrix, dense_vector)

56.6 µs ± 203 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [33]:
%timeit torch.mv(csr_matrix, dense_vector)

12.1 µs ± 0.351 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [35]:
%timeit torch.mv(dense_matrix, dense_vector)

254 µs ± 11.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
